In [36]:

from PIL import Image
import numpy as np
import cv2
import time
import sys

start = time.time()

%run "common_functions.py"

sys.setrecursionlimit(53000) # override needed for computing midpoints, which uses a recursive function
Image.MAX_IMAGE_PIXELS = 366498276 # override is needed, or else it gives a DecompressionBombError

cy1_shifted_file = "originals/shiftedcycle 1thnksgv.tif"
cy2_shifted_file = "originals/shiftedcycle 2thnksgv.tif"

def c(img):
    return Image.fromarray(img)

def overlay(arr1, arr2):
    return (arr1 / 3 + arr2 / 2);

In [37]:
###  #  #  #  #  #  #  #  #  ###
#                              #
#      LOADING THE IMAGES      #
#                              #
###  #  #  #  #  #  #  #  #  ###

layer_to_align = 2

img_cy1 = Image.open(cy1_shifted_file)
brightfield_cy1 = np.array(img_cy1)
brightfield_cy1 = (brightfield_cy1/256).astype('uint8') # don't need the whole range of values, so this reduces mem size, + improves access speed

img_cy2 = Image.open(cy2_shifted_file)
brightfield_cy2 = np.array(img_cy2)
brightfield_cy2 = (brightfield_cy2/256).astype('uint8') 

img_cy1 = None
img_cy2 = None

brightfield_cy1[brightfield_cy2 == 0] = 0
brightfield_cy2[brightfield_cy1 == 0] = 0

print(brightfield_cy2.shape)

(15640, 15640)


In [38]:
###  #  #  #  #  #  #  #  #  ###
#                              #
# SLICING SECTION FOR ANALYSIS #
#                              #
###  #  #  #  #  #  #  #  #  ###

START = 0
SIZE_OF_ANALYSIS = 15640
if brightfield_cy1.shape[0] > SIZE_OF_ANALYSIS: 
    brightfield_cy1 = brightfield_cy1[START:START+SIZE_OF_ANALYSIS, START:START+SIZE_OF_ANALYSIS]
    brightfield_cy2 = brightfield_cy2[START:START+SIZE_OF_ANALYSIS, START:START+SIZE_OF_ANALYSIS]
print(brightfield_cy1.shape)

# c(brightfield_cy2)

(15640, 15640)


In [39]:
def blockify(cuts):
    
    # Returns the center points of all the blocks in a cuts x cuts array
    
    centerpoints = []
    for i in range(cuts):
        row = []
        for j in range(cuts):
            # print((i + 1), cuts, (j + 1), cuts)
            row.append(np.array([(2*i + 1) / (cuts *2), (2*j + 1) / (cuts *2)]))
            # print((2*i + 1) / (cuts *2))
        
        centerpoints.append(np.array(row))
            
    return np.array(centerpoints)

def alignImages(im1, im2):
  # Convert images to grayscale
  #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(10000)
  keypoints1, descriptors1 = orb.detectAndCompute(im1, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2, None)
 
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = list(matcher.match(descriptors1, descriptors2, None))
 
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  print("Count of Matches:", len(matches))
  numGoodMatches = int(len(matches) * .01)
  print("Count of Good Matches:", numGoodMatches)
  print("Distance of worst match:", matches[numGoodMatches].distance)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  #imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  #cv2.imwrite("matches.jpg", imMatches)
 
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
 
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width = np.array(im2.shape) * 1.5
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
 
  return im1Reg, h


In [45]:
###  #  #  #  #  #  #  #  #  ###
#                              #
#          SEE CUTS!!          #
#                              #
###  #  #  #  #  #  #  #  #  ###


# https://factorization.info/factors/1/factors-of-17472.html 
# the second param as below can be
num_quads = int(brightfield_cy1.shape[0] / 550)

num_quads = 3



block_points = blockify(num_quads) * brightfield_cy1.shape[0]
width = brightfield_cy1.shape[0] / 32 / 2
# print( block_points )
# lol = brightfield_cy2.copy()
# for i in block_points:
#     for j in i:
#         # print(j)
#         j = j.astype(int)
        
#         lol = cv2.circle(lol, j.astype(int), 10, 255, 10)
#         x,y = j
#         girth = int(block_points[1][1][0])
#         cv2.rectangle(lol, (x-girth, y+girth), (x+girth, y-girth), (255, 255, 255), 2) 
        # print((x-girth, y-girth), (x+girth, y+girth))
  
# c(lol)

In [46]:
import mul
import importlib
importlib.reload(mul)

print(dir(mul))

['Image', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'alignImages', 'c', 'cv2', 'do_', 'get_overlain', 'np', 'os', 'sys', 'time']


In [47]:

new_crew = c(np.zeros((SIZE_OF_ANALYSIS,SIZE_OF_ANALYSIS))).convert("RGBA")

processes = []
import concurrent.futures
with concurrent.futures.ProcessPoolExecutor() as executor: 
    for i in block_points:
        for j in i:
            x,y = j.astype(int)
            print(x,y)
            # image_set = 
            processes.append(executor.submit(mul.get_overlain, brightfield_cy1[y-girth:y+girth, x- girth:x+girth,], brightfield_cy2[y-girth:y+girth, x- girth:x+girth,], x, y))
        

    




# new_crew.save("nc.png")

# new_crew

2606 2606
2606 7820
2606 13033
7820 2606
7820 7820
7820 13033
13033 2606
13033 7820
13033 13033
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 49.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 49.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 41.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 54.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 52.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 46.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 48.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 53.0
Count of Matches: 10000
Count of Good Matches: 100
Distance of worst match: 48.0


In [48]:
for p in processes:
    xy, imgAligned = p.result()
    x, y = xy
    print(x, y, imgAligned)
    new_crew.paste(imgAligned,(x-girth, y-girth), imgAligned)


2606 2606 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159B0ACD0>
2606 7820 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x14846AF50>
2606 13033 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159CD2510>
7820 2606 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159B92E50>
7820 7820 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159CE0690>
7820 13033 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159B93310>
13033 2606 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159BBABD0>
13033 7820 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159BBB390>
13033 13033 <PIL.Image.Image image mode=RGBA size=7818x7818 at 0x159CE3950>


In [49]:
c(brightfield_cy2).save("orig_3x3.png")
print("first saved!")
(new_crew.convert("L")).save("shif_3x3.png")

print(time.time() - start)

first saved!
145.31757616996765
